<a href="https://colab.research.google.com/github/lalitha1201/AutoRecommender/blob/master/Insights_Auto_Rec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Recommendation engine using collaborating filtering on Movielens**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [9]:
!pip install surprise

     |████████████████████████████████| 6.5MB 2.8MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678236 sha256=b1cde3ae390f87562593a84fa9a134f0a6f6e7ab8903a30a5d914b5505e9f04a
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import torch

In [14]:
!pip install fastai==0.7.0

     |████████████████████████████████| 122kB 4.8MB/s 
     |████████████████████████████████| 496.4MB 32kB/s 
     |████████████████████████████████| 1.5MB 45.4MB/s 
  Created wheel for bcolz: filename=bcolz-1.2.1-cp36-cp36m-linux_x86_64.whl size=2661288 sha256=8364667cbc27442eef0e477d2bc9d380aaa832971ce78cc220603f7e7e171f89
  Stored in directory: /root/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
Successfully built bcolz
ERROR: torchvision 0.4.2 has requirement torch==1.3.1, but you'll have torch 0.3.1 which is incompatible.
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1
  Found existing installation: fastai 1.0.60
    Uninstalling fastai-1.0.60:
      Successfully uninstalled fastai-1.0.60


In [0]:
!pip install torchtext==0.2.3

In [1]:
!pip install torchvision

     |████████████████████████████████| 734.6MB 22kB/s 
ERROR: fastai 0.7.0 has requirement torch<0.4, but you'll have torch 1.3.1 which is incompatible.
  Found existing installation: torch 0.3.1
    Uninstalling torch-0.3.1:
      Successfully uninstalled torch-0.3.1


In [0]:
import torch
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd

from fastai.learner import *
from fastai.column_data import *

In [0]:
ratings = pd.read_csv('gdrive/My Drive/ml-latest-small/ratings.csv')

In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [0]:
ratings.shape

(100836, 4)

In [0]:
n_users=int(ratings.userId.nunique())
n_movies=int(ratings.movieId.nunique())

In [0]:
print("n_users = ",n_users, "||", "n_movies = ", n_movies )

n_users =  610 || n_movies =  9724


In [0]:
g = ratings.groupby('userId')['rating'].count()
topg = g.sort_values(ascending = False)[:15]

i = ratings.groupby('movieId')['rating'].count()
topi = i.sort_values(ascending = False)[:15]


In [0]:
import numpy as np


In [0]:
# gettings ratings of top users and top items


join1 = ratings.join(topg, on='userId', how = 'inner', rsuffix='_r')
join1 = join1.join(topi, on='movieId', how = 'inner', rsuffix = '_r')

pd.crosstab(join1.userId, join1.movieId, join1.rating, aggfunc=np.sum)

movieId,1,50,110,260,296,318,356,480,527,589,593,1196,2571,2858,2959
userId,,,,,,,,,,,,,,,
68,2.5,3.0,2.5,5.0,2.0,3.0,3.5,3.5,4.0,3.5,3.5,5.0,4.5,5.0,2.5
182,4.0,4.5,3.5,3.5,5.0,4.5,5.0,3.5,4.0,2.0,4.5,3.0,5.0,5.0,5.0
249,4.0,4.0,5.0,5.0,4.0,4.5,4.5,4.0,4.5,4.0,4.0,5.0,5.0,4.5,5.0
274,4.0,4.0,4.5,3.0,5.0,4.5,4.5,3.5,4.0,4.5,4.0,4.5,4.0,5.0,5.0
288,4.5,NaN,5.0,5.0,5.0,5.0,5.0,2.0,5.0,4.0,5.0,4.5,3.0,NaN,3.5
307,4.0,4.5,3.5,3.5,4.5,4.5,4.0,3.5,4.5,2.5,4.5,3.0,3.5,4.0,4.0
380,5.0,4.0,4.0,5.0,5.0,3.0,5.0,5.0,NaN,5.0,5.0,5.0,4.5,NaN,4.0
387,NaN,4.5,3.5,4.5,5.0,3.5,4.0,3.0,NaN,3.5,4.0,4.5,4.0,4.5,4.5
414,4.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0,4.0,5.0,5.0,5.0,5.0


# Collaborative filtering

In [0]:
val_indx = get_cv_idxs(len(ratings))  # index for validation set
wd = 2e-4 # weight decay
n_factors = 50 # n_factors

In [0]:
cf = CollabFilterDataset.from_csv('gdrive/My Drive/ml-latest-small', 'ratings.csv', 'userId', 'movieId', 'rating')

In [0]:
learn = cf.get_learner(n_factors, val_indx, bs=64, opt_fn=optim.Adam)

In [0]:
learn.fit(1e-2,2, wds = wd, cycle_len=1, cycle_mult=2)

  1%|          | 15/1261 [00:00<00:18, 69.09it/s, loss=1.63]

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch      trn_loss   val_loss   
    0      0.783299   0.774151  
    1      0.733051   0.754413  
    2      0.588378   0.735052  



[0.7350520817464573]

# Collaborating filter from scratch

In [0]:
u_uniq = ratings.userId.unique()
user2idx = {o:i for i,o in enumerate(u_uniq)}
ratings.userId = ratings.userId.apply(lambda x: user2idx[x])

m_uniq = ratings.movieId.unique()
movie2idx = {o:i for i,o in enumerate(m_uniq)}
ratings.movieId = ratings.movieId.apply(lambda x: movie2idx[x])

In [0]:
n_users, n_movies

(610, 9724)

In [0]:
val_indx = get_cv_idxs(len(ratings))  # index for validation set
wd = 2e-4 # weight decay
n_factors = 50 # n_factors i.e. 1 dimension of embeddings (random)

In [0]:
min_rating,max_rating = ratings.rating.min(),ratings.rating.max()
min_rating,max_rating

(0.5, 5.0)

In [0]:
def get_emb(ni,nf):
    e = nn.Embedding(ni, nf)
    e.weight.data.uniform_(-0.01,0.01)
    #e.weight.data.normal_(0,0.003)

    return e

In [0]:
x = ratings.drop(['rating'],axis=1)
y = ratings['rating'].astype(np.float32)

data = ColumnarModelData.from_data_frame('gdrive/My Drive/ml-latest-small', val_indx, x, y, ['userId', 'movieId'], 64)

In [0]:
# nh = dimension of hidden linear layer
# p1 = dropout1
# p2 = dropout2

class EmbeddingNet(nn.Module):
    def __init__(self, n_users, _n_movies, nh = 10, p1 = 0.05, p2= 0.5):
        super().__init__()
        (self.u, self.m, self.ub, self.mb) = [get_emb(*o) for o in [
            (n_users, n_factors), (n_movies, n_factors),
            (n_users,1), (n_movies,1)
        ]]
        
        self.lin1 = nn.Linear(n_factors*2, nh)  # bias is True by default
        self.lin2 = nn.Linear(nh, 1)
        self.drop1 = nn.Dropout(p = p1)
        self.drop2 = nn.Dropout(p = p2)
    
    def forward(self, cats, conts): # forward pass i.e.  dot product of vector from movie embedding matrixx
                                    # and vector from user embeddings matrix
        
        # torch.cat : concatenates both embedding matrix to make more columns, same rows i.e. n_factors*2, n : rows
        # u(users) is doing lookup for indexed mentioned in users
        # users has indexes to lookup in embedding matrix. 
        
        users,movies = cats[:,0],cats[:,1]
        u2,m2 = self.u(users) , self.m(movies)
       
        x = self.drop1(torch.cat([u2,m2], 1)) # drop initialized weights
        x = self.drop2(F.relu(self.lin1(x))) # drop 1st linear + nonlinear wt
        r = F.sigmoid(self.lin2(x)) * (max_rating - min_rating) + min_rating               
        return r

In [0]:
wd=1e-5
model = EmbeddingNet(n_users, n_movies)
#model = model.cuda()
opt = optim.Adam(model.parameters(), 1e-3, weight_decay=wd) # got parameter() for free , lr = 1e-3

In [0]:
model

EmbeddingNet(
  (u): Embedding(610, 50)
  (m): Embedding(9724, 50)
  (ub): Embedding(610, 1)
  (mb): Embedding(9724, 1)
  (lin1): Linear(in_features=100, out_features=10, bias=True)
  (lin2): Linear(in_features=10, out_features=1, bias=True)
  (drop1): Dropout(p=0.05, inplace=False)
  (drop2): Dropout(p=0.5, inplace=False)
)

In [0]:

fit(model, data, 3, opt, F.mse_loss)

  1%|▏         | 17/1261 [00:00<00:16, 75.58it/s, loss=1.95]

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch      trn_loss   val_loss   
    0      0.850896   0.788031  
    1      0.780398   0.767577  
    2      0.732759   0.76894   



[0.7689395845039815]

In [0]:
set_lrs(opt, 1e-3)

In [0]:
fit(model, data, 3, opt, F.mse_loss)

  0%|          | 6/1261 [00:00<00:21, 57.89it/s, loss=0.747]

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch      trn_loss   val_loss   
    0      0.742704   0.769719  
    1      0.722744   0.768911  
    2      0.706393   0.771855  



[0.7718545336982116]

In [0]:
from surprise import Reader, Dataset
# Define the format

reader = Reader(line_format='user item rating timestamp', sep='\t')
# Load the data from the file using the reader format

data = Dataset.load_from_file('gdrive/My Drive/ml-100k/u.data', reader=reader)

In [0]:
ratings[:2]

,userId,movieId,rating,timestamp
0,0,0,4.0,964982703
1,0,1,4.0,964981247


In [0]:
ratings_dict = {'itemID': list(ratings.movieId),
                'userID': list(ratings.userId),
                'rating': list(ratings.rating)}
df = pd.DataFrame(ratings_dict)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0.5, 5.0))
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

In [13]:
# train SVD on 75% of known rates
trainset, testset = train_test_split(data, test_size=.25)
algorithm = SVD()
algorithm.fit(trainset)
predictions = algorithm.test(testset)

# check the accuracy using Root Mean Square Error
accuracy.rmse(predictions)

RMSE: 0.8769


0.8769274006843453

In [0]:
def pred_user_rating(ui):
    if ui in ratings_f.userId.unique():
        ui_list = ratings_f[ratings_f.userId == ui].movieId.tolist()
        d = {k: v for k,v in Mapping_file.items() if not v in ui_list}        
        predictedL = []
        for i, j in d.items():     
            predicted = algorithm.predict(ui, j)
            predictedL.append((i, predicted[3])) 
        pdf = pd.DataFrame(predictedL, columns = ['movies', 'ratings'])
        pdf.sort_values('ratings', ascending=False, inplace=True)  
        pdf.set_index('movies', inplace=True)    
        return pdf.head(10)        
    else:
        print("User Id does not exist in the list!")
        return None

In [15]:
user_id = 7020
pred_user_rating(user_id)

NameError: ignored

In [0]:
# Split data into 5 folds

#data.split(n_folds=5)

AttributeError: ignored

# Surprise package

In [0]:
!pip install surprise

# SVD

In [8]:
from surprise import SVD
from surprise.model_selection import cross_validate

ModuleNotFoundError: ignored

In [0]:

from surprise.model_selection import GridSearchCV


In [0]:
param_grid = {'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
grid_search = GridSearchCV(SVD, param_grid, measures=['RMSE'])
grid_search.fit(data)

# best RMSE score
print(grid_search.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(grid_search.best_params['rmse'])

0.8826561113722182
{'lr_all': 0.005, 'reg_all': 0.4}


https://surprise.readthedocs.io/en/stable/getting_started.html

In [0]:

# We'll use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8836  0.8754  0.8727  0.8697  0.8716  0.8746  0.0049  
MAE (testset)     0.6775  0.6726  0.6736  0.6676  0.6704  0.6723  0.0033  
Fit time          5.35    5.38    5.27    5.39    5.48    5.37    0.07    
Test time         0.14    0.25    0.15    0.15    0.14    0.17    0.04    


{'fit_time': (5.352824687957764,
  5.376279830932617,
  5.266646862030029,
  5.394549608230591,
  5.476244211196899),
 'test_mae': array([0.67754, 0.67259, 0.6736 , 0.66759, 0.67042]),
 'test_rmse': array([0.88363, 0.87544, 0.87267, 0.86968, 0.87162]),
 'test_time': (0.14324069023132324,
  0.24808025360107422,
  0.14529085159301758,
  0.14739251136779785,
  0.1432640552520752)}

# KNN algorithm

In [0]:
from surprise import KNNBasic
from surprise import Dataset

# Load the movielens-100k dataset
#data = Dataset.load_builtin('ml-100k')

# Retrieve the trainset.
trainset = data.build_full_trainset()

# Build an algorithm, and train it.
algo = KNNBasic()
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [0]:
uid = str(196)  # raw user id (as in the ratings file). They are **strings**!
iid = str(302)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

user: 196        item: 302        r_ui = 4.00   est = 3.50   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}


In [0]:
from surprise import BaselineOnly


In [0]:
cross_validate(BaselineOnly(), data, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8744  0.8758  0.8700  0.8740  0.8697  0.8728  0.0025  
MAE (testset)     0.6727  0.6753  0.6734  0.6737  0.6691  0.6729  0.0021  
Fit time          0.26    0.25    0.26    0.25    0.19    0.24    0.03    
Test time         0.21    0.09    0.21    0.09    0.08    0.14    0.06    


{'fit_time': (0.2623324394226074,
  0.2469778060913086,
  0.2605245113372803,
  0.2513587474822998,
  0.19013547897338867),
 'test_mae': array([0.67275, 0.67532, 0.6734 , 0.67373, 0.66911]),
 'test_rmse': array([0.87441, 0.87578, 0.87004, 0.87399, 0.86969]),
 'test_time': (0.2077643871307373,
  0.08873724937438965,
  0.21107125282287598,
  0.08643054962158203,
  0.08424139022827148)}

In [0]:

from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate


# Creation of the dataframe. Column names are irrelevant.
ratings_dict = {'itemID': [1, 1, 1, 2, 2],
                'userID': [9, 32, 2, 45, 'user_foo'],
                'rating': [3, 2, 4, 3, 1]}
df = pd.DataFrame(ratings_dict)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(NormalPredictor(), data, cv=2)

{'fit_time': (0.0004374980926513672, 7.987022399902344e-05),
 'test_mae': array([0.90873, 0.7628 ]),
 'test_rmse': array([0.93396, 0.78011]),
 'test_time': (0.00010037422180175781, 5.626678466796875e-05)}

In [0]:
from surprise.model_selection import KFold
from surprise import accuracy
kf = KFold(n_splits=3)

algo = SVD()

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 1.8870
RMSE: 0.6739
RMSE: 1.7079


In [0]:
predictions = algo.fit(trainset).test(testset)

# NMF

In [0]:
from surprise import NMF

In [0]:
algo = NMF()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.5000  0.5000  0.7500  2.0000  1.7500  1.1000  0.6442  
MAE (testset)     0.5000  0.5000  0.7500  2.0000  1.7500  1.1000  0.6442  
Fit time          0.01    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'fit_time': (0.00769805908203125,
  0.000982522964477539,
  0.0009407997131347656,
  0.00098419189453125,
  0.0011191368103027344),
 'test_mae': array([0.5 , 0.5 , 0.75, 2.  , 1.75]),
 'test_rmse': array([0.5 , 0.5 , 0.75, 2.  , 1.75]),
 'test_time': (5.936622619628906e-05,
  2.6464462280273438e-05,
  2.1457672119140625e-05,
  2.3365020751953125e-05,
  4.5299530029296875e-05)}

In [0]:
ratings = pd.read_csv('gdrive/My Drive/ml-latest-small/ratings.csv')

In [0]:
ratings[:2]

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


In [0]:
ratings2 = ratings.copy()

In [0]:
col = ['movieId', 'userId']

In [0]:
for c in col:
    ratings2[c].replace({val: i for i, val in enumerate(ratings2[c].unique())}, inplace=True)

In [0]:

ratings2[:2]

,userId,movieId,rating,timestamp
0,0,0,4.0,964982703
1,0,1,4.0,964981247


In [0]:
n_users=int(ratings2.userId.nunique())
n_items=int(ratings2.movieId.nunique())

In [0]:
print("n_users = ",n_users, "||", "n_items = ", n_items )

n_users =  610 || n_items =  9724


In [0]:
!pip install sklearn

In [0]:
from sklearn.model_selection import cross_validate as cv
train_data, test_data = cv.train_test_split(ratings2, test_size=0.25)

AttributeError: ignored

In [0]:
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split

In [7]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


In [0]:
surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

In [9]:
surprise_results

,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.925632,119.255502,5.515756
KNNBaseline,0.936168,0.328846,6.204396
SVD,0.944697,4.048044,0.319655
BaselineOnly,0.946929,0.124158,0.214374
SlopeOne,0.950232,0.515903,4.221160
KNNWithZScore,0.956059,0.293822,5.753429
KNNWithMeans,0.956573,0.245670,5.378050
NMF,0.973999,4.302225,0.250262
CoClustering,0.974002,1.233386,0.290061


# Get Top N predictions

In [10]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
196 ['127', '64', '408', '357', '603', '169', '185', '187', '318', '483']
186 ['408', '69', '483', '513', '511', '516', '427', '478', '64', '197']
22 ['64', '313', '223', '169', '114', '98', '652', '182', '923', '199']
244 ['134', '127', '199', '483', '474', '408', '192', '98', '520', '589']
166 ['483', '515', '318', '513', '178', '98', '211', '484', '199', '480']
298 ['64', '190', '48', '408', '923', '493', '520', '12', '963', '316']
115 ['135', '134', '408', '285', '156', '603', '86', '42', '606', '14']
253 ['272', '178', '174', '651', '313', '169', '513', '143', '423', '408']
305 ['316', '57', '513', '498', '661', '211', '515', '194', '132', '657']
6 ['603', '654', '657', '114', '179', '428', '922', '661', '1524', '659']
62 ['169', '223', '187', '427', 

In [16]:
from collections import defaultdict

from surprise import Dataset
from surprise import SVD
from surprise.model_selection import KFold


def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls


data = Dataset.load_builtin('ml-100k')
kf = KFold(n_splits=5)
algo = SVD()

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.8806942968473254
0.2566726732082153
0.8775300778485492
0.2543223298677209
0.8786270346779901
0.25437995297526134
0.8732199787460144
0.25736873899619095
0.8622918880623445
0.2581333868254998


In [17]:
import io  # needed because of weird encoding of u.item file

from surprise import KNNBaseline
from surprise import Dataset
from surprise import get_dataset_dir


def read_item_names():
    """Read the u.item file from MovieLens 100-k dataset and return two
    mappings to convert raw ids into movie names and movie names into raw ids.
    """

    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid


# First, train the algortihm to compute the similarities between items
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
sim_options = {'name': 'pearson_baseline', 'user_based': False}
algo = KNNBaseline(sim_options=sim_options)
algo.fit(trainset)

# Read the mappings raw id <-> movie name
rid_to_name, name_to_rid = read_item_names()

# Retrieve inner id of the movie Toy Story
toy_story_raw_id = name_to_rid['Toy Story (1995)']
toy_story_inner_id = algo.trainset.to_inner_iid(toy_story_raw_id)

# Retrieve inner ids of the nearest neighbors of Toy Story.
toy_story_neighbors = algo.get_neighbors(toy_story_inner_id, k=10)

# Convert inner ids of the neighbors into names.
toy_story_neighbors = (algo.trainset.to_raw_iid(inner_id)
                       for inner_id in toy_story_neighbors)
toy_story_neighbors = (rid_to_name[rid]
                       for rid in toy_story_neighbors)

print()
print('The 10 nearest neighbors of Toy Story are:')
for movie in toy_story_neighbors:
    print(movie)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.

The 10 nearest neighbors of Toy Story are:
Beauty and the Beast (1991)
Raiders of the Lost Ark (1981)
That Thing You Do! (1996)
Lion King, The (1994)
Craft, The (1996)
Liar Liar (1997)
Aladdin (1992)
Cool Hand Luke (1967)
Winnie the Pooh and the Blustery Day (1968)
Indiana Jones and the Last Crusade (1989)


# Amazon Reviews Dataset

In [0]:
import pandas as pd

In [0]:
data1 = pd.read_csv('gdrive/My Drive/sample_us.tsv',sep='\t',encoding='latin-1')

In [7]:
data1.head()

bplist00Ñ_WebMainResourceÕ   Unnamed: 1


_WebResourceTextEncodingName^WebResourceURL... customer_id review_id      product_id product_parent product_title                                      product_category star_rating helpful_votes total_votes vine verified_purchase review_headline                                                                           review_body  review_date
US                                                 18778586    RDIJS7QYB6XNR  B00EDBY7X8 122952789      Monopoly Junior Board Game                         Toys             5           0             0           N    Y                 Five Stars                                                                               Excellent!!!   2015-08-31
                                                   24769659    R36ED1U38IELG8 B00D7JFOPC 952062646      56 Pieces of Wooden Train Track Compatible with... Toys             5           0             0           N    Y                 Good quality track at excellent price               Great quality wooden track (better than some o...   2015-08-31
                                                   44331596    R1UE3RPRGCOLD  B002LHA74O 818126353      Super Jumbo Playing Cards by S&amp;S Worldwide     Toys             2           1             1           N    Y                 Two Stars                                                           Cards are not as big as pictured.   2015-08-31
                                                   23310293    R298788GS6I901 B00ARPLCGY 261944918      Barbie Doll and Fashions Barbie Gift Set           Toys             5           0             0           N    Y                 my daughter loved it and i liked the price and ...  my daughter loved it and i liked the price and...   2015-08-31